In [0]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [817 kB]
Get:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/ma

In [0]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

--2020-02-09 11:22:12--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.68.26, 52.206.140.199, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [               <=>  ]   1.55G  3.20MB/s    in 8m 29s  

2020-02-09 11:30:43 (3.12 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1665791877]



In [0]:
!mv rows.csv\?accessType\=DOWNLOAD reported_crimes.csv

In [0]:
df=spark.read.csv('reported_crimes.csv',header=True)

In [0]:
df.createOrReplaceTempView(name='reported_crimes')

In [0]:
sample=df.limit(50)

In [0]:
sample.cache()
sample.count()

50

In [0]:
sample.show(1)

+--------+-----------+--------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|                Date|          Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+--------------------+---------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11034701|   JA366925|01/01/2001 11:00:...|016XX E 86TH PL|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|        null|        

In [0]:
import pyspark.sql.functions as F
sample=sample.withColumn('Arrested',F.when(sample.Arrest=='false',"No"))

In [0]:
sample.show(2)

+--------+-----------+--------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+--------+
|      ID|Case Number|                Date|             Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|Arrested|
+--------+-----------+--------------------+------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+--------+
|11034701|   JA366925|01/01/2001 11:00:...|   016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|  

In [0]:
sample=df.limit(20)

In [0]:
sample.cache()
sample.count()

20

In [0]:
sample.select('Location Description').distinct().show()

+--------------------+
|Location Description|
+--------------------+
|           RESIDENCE|
|               OTHER|
|         HOTEL/MOTEL|
|                null|
|           APARTMENT|
|            SIDEWALK|
|PARKING LOT/GARAG...|
|    RESIDENCE-GARAGE|
+--------------------+



In [0]:
from pyspark.sql.functions import col

In [0]:
sample=sample.withColumn('something',F.when(col('Location Description')=='OTHER','Someplace').when(col('Location Description')!='OTHER','I know'))

In [0]:
sample.select(col('Location Description'),col('something')).show()

+--------------------+---------+
|Location Description|something|
+--------------------+---------+
|           RESIDENCE|   I know|
|           RESIDENCE|   I know|
|               OTHER|Someplace|
|           RESIDENCE|   I know|
|         HOTEL/MOTEL|   I know|
|           RESIDENCE|   I know|
|           RESIDENCE|   I know|
|           RESIDENCE|   I know|
|           RESIDENCE|   I know|
|                null|     null|
|           APARTMENT|   I know|
|           APARTMENT|   I know|
|           RESIDENCE|   I know|
|            SIDEWALK|   I know|
|           APARTMENT|   I know|
|               OTHER|Someplace|
|PARKING LOT/GARAG...|   I know|
|           RESIDENCE|   I know|
|           RESIDENCE|   I know|
|    RESIDENCE-GARAGE|   I know|
+--------------------+---------+



In [0]:
sample.select('*').where(col('Location Description').isNull()).show()

+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---------+
|      ID|Case Number|                Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|something|
+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---------+
|11227633|   JB147500|12/28/2017 03:55:...|011XX S MICHIGAN AVE|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                null| false|   false|0123|     001|

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

sample=sample.withColumn('ID',monotonically_increasing_id())

In [0]:
sample.orderBy(sample.ID.desc()).show(10)

+---+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---------+
| ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|something|
+---+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---------+
| 19|   JA375398|07/29/2017 10:00:...|022XX W WINNEMAC AVE|0810|               THEFT|           OVER $500|    RESIDENCE-GARAGE| false|   false|2031|     020|  47|       

In [0]:
sample.orderBy(sample.ID,ascending=False).show(10)

+---+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---------+
| ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|something|
+---+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---------+
| 19|   JA375398|07/29/2017 10:00:...|022XX W WINNEMAC AVE|0810|               THEFT|           OVER $500|    RESIDENCE-GARAGE| false|   false|2031|     020|  47|       